In [20]:
import os, json
from pathlib import Path
from dotenv import load_dotenv
from typing import Dict, Any, List
from pprint import pprint

from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_upstage import UpstageEmbeddings

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities import WikipediaAPIWrapper

In [21]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

DATA_PATH = Path("./data/cafe_menu.txt")
DB_PATH = Path("./db/cafe_db")
DB_PATH.parent.mkdir(parents=True, exist_ok=True)

In [22]:
if not DATA_PATH.exists():
    raise FileNotFoundError(f"{DATA_PATH} 파일이 없습니다. ./data/ 경로를 확인하세요.")

file_content = DATA_PATH.read_text(encoding="utf-8")

# 메뉴 항목은 빈 줄(\n\n)로 구분되어 있다고 가정
menu_items = [blk.strip() for blk in file_content.strip().split("\n\n") if blk.strip()]

# 원문 단락 자체를 문서화(간단하게 유지)
docs = [Document(page_content=item, metadata={"source": "cafe_menu"}) for item in menu_items]

# 필요 시 문장 길이 보정을 위해 간단 split (선택)
splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=0)
docs = splitter.split_documents(docs)

upstage_key = os.getenv("UPSTAGE_API_KEY")
if not upstage_key:
    raise RuntimeError("UPSTAGE_API_KEY가 필요합니다 (.env 확인).")
embeddings = UpstageEmbeddings(model="solar-embedding-1-large", api_key=upstage_key)
db = FAISS.from_documents(docs, embeddings)
db.save_local(str(DB_PATH))

In [29]:
# ----------------------------------------------------------------------
# 2) 도구 정의 (tavily / wikipedia / local-db search)
# ----------------------------------------------------------------------
# a) Tavily (웹 최신 검색) — LangChain의 TavilySearchResults 툴
tavily_search_func = TavilySearchResults(max_results=2, name="tavily_search")

# b) Wikipedia 요약
@tool
def wiki_summary(query: str) -> str:
    """위키피디아에서 일반 지식을 검색/요약합니다."""
    wikipedia = WikipediaAPIWrapper(lang="ko", top_k_results=1, doc_content_chars_max=2000)
    try:
        return wikipedia.run(query)
    except Exception:
        return f"위키에서 '{query}' 정보를 찾지 못했습니다."

# c) 로컬 카페 메뉴 DB 검색
@tool
def db_search_cafe_func(query: str) -> str:
    """로컬 카페 메뉴 DB에서 가격/재료/설명 관련 정보를 검색합니다."""
    vector_db = FAISS.load_local(str(DB_PATH), embeddings, allow_dangerous_deserialization=True)
    retriever = vector_db.as_retriever(search_kwargs={"k": 2})
    # 최신 API는 invoke 권장
    results = retriever.invoke(query)
    if not results:
        return "관련 메뉴 정보를 찾을 수 없습니다."
    lines = []
    for d in results:
        lines.append(d.page_content.replace("\n", " "))
    return "메뉴 검색 결과:\n" + "\n".join(lines)

# ----------------------------------------------------------------------
# 3) LLM + 툴 바인딩 & 체인 실행
# ----------------------------------------------------------------------
tools = [tavily_search_func, wiki_summary, db_search_cafe_func]

llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0
)

def run_tool_chain(user_input: str) -> str:
    # 1차 호출: 어떤 툴을 쓸지 LLM이 결정
    first = llm.invoke([HumanMessage(content=user_input)])

    # 툴 호출이 없으면 바로 답변
    if not isinstance(first, AIMessage) or not getattr(first, "tool_calls", None):
        return first.content

    tool_msgs: List[ToolMessage] = []
    print("\n--- LLM tool_calls ---")
    pprint(first.tool_calls)

    # 각 툴 실행
    for call in first.tool_calls:
        name: str = call["name"]
        tool_id: str = call["id"]
        args: Any = call.get("args", {})

        # args가 str로 오는 모델도 있어 안전 처리
        if isinstance(args, str):
            try:
                args = json.loads(args)
            except json.JSONDecodeError:
                args = {}

        # 공통적으로 'query' 키 사용
        if name == "tavily_search":
            out = tavily_search_func.invoke(args.get("query", user_input))
        elif name == "wiki_summary":
            out = wiki_summary.invoke(args.get("query", user_input))
        elif name == "db_search_cafe_func":
            out = db_search_cafe_func.invoke(args.get("query", user_input))
        else:
            out = f"알 수 없는 도구: {name}"

        tool_msgs.append(ToolMessage(content=str(out), tool_call_id=tool_id))

    # 도구 결과를 다시 LLM에 전달해 최종 답변 생성
    final = llm.invoke([HumanMessage(content=user_input), first, *tool_msgs])
    return final.content

# ----------------------------------------------------------------------
# 4) 테스트
# ----------------------------------------------------------------------
if __name__ == "__main__":
    q = "아메리카노의 가격과 특징은 무엇인가요?"
    ans = run_tool_chain(q)
    print(f"\n[질문] {q}")
    print(f"[답변] {ans}")


[질문] 아메리카노의 가격과 특징은 무엇인가요?
[답변] 아메리카노는 **에스프레소에 뜨거운 물을 더해 만든 커피**로, **진한 커피 맛**과 **간단한 구성**이 특징입니다.

---

### ✅ **특징**
| 항목 | 설명 |
|------|------|
| **맛** | 에스프레소의 진한 맛이 느껴지지만, 물로 희석돼 **쓴맛이 강한 편**입니다. |
| **칼로리** | **0kcal** (설탕, 시럽 없이 마실 경우) |
| **카페인** | 보통 **1샷(75~100mg)** 기준, 2샷도 가능 |
| **구성** | 에스프레소 + 뜨거운 물 (우유, 설탕 없음) |
| **ICE 아메리카노** | 에스프레소 + 시원한 물 + 얼음 |

---

### 💰 **가격 (2025년 기준, 서울 기준)**
| 브랜드 | Hot/Ice | 가격 |
|--------|----------|--------|
| **스타벅스** | Tall 사이즈 | ₩4,600 |
| **메가커피** | 기본 | ₩1,500 |
| **커피빈** | Tall | ₩4,100 |
| **할리스** | Tall | ₩4,200 |
| **이디야** | 기본 | ₩3,500 |
| **커피 전문점(일반)** | – | ₩2,000~₩4,500 |

---

### 🔍 요약
- **가장 단순하면서도 깔끔한 커피**
- **칼로리 걱정 없이 카페인 섭취 가능**
- **가격은 브랜드에 따라 1,500원~4,600원으로 큰 차이**

궁금한 브랜드나 메뉴가 더 있다면 말씀해 주세요.
